<a href="https://colab.research.google.com/github/sanamehrabbeigi/Undergrad-Recommendation-System/blob/main/JobRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hello')

hello


**Implementation of Recommendation Systems using ML Algorithms**	                                                                                                              
Implemented a recommendation system using machine learning techniques, including collaborative filtering, matrix factorization, neural networks, and singular value decomposition (SVD) on the Kaggle job recommendation challenge dataset.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
apps = pd.read_csv('./job-recommendation/apps.tsv', delimiter='\t')
pop_jobs = pd.read_csv('./job-recommendation/popular_jobs.csv')
users = pd.read_csv('./job-recommendation/users.tsv', delimiter='\t')
user_history = pd.read_csv('./job-recommendation/user_history.tsv', delimiter='\t')
window_dates = pd.read_csv('./job-recommendation/window_dates.tsv', delimiter='\t')


In [ ]:
jobs, count = np.unique(apps.JobID.to_numpy(), return_counts=True)
popular_jobs = jobs[count.argsort()[-1000:]]

In [ ]:
new_apps = apps.loc[np.isin(apps.JobID, popular_jobs)].copy()
uniqs = new_apps.UserID.unique()
uniqs_job = new_apps.JobID.unique()

In [ ]:
uniqs_assign = {uniqs[i]: i for i in range(uniqs.shape[0])}
reassigned_users = [uniqs_assign[a] for a in new_apps.UserID]

uniqs_assign_job = {uniqs_job[i]: i for i in range(uniqs_job.shape[0])}
reassigned_jobs = [uniqs_assign_job[a] for a in new_apps.JobID]


In [ ]:
new_apps['JobID'] = reassigned_jobs
new_apps['UserID'] = reassigned_users

In [ ]:
new_apps = new_apps.drop(['WindowID','Split','ApplicationDate'], axis=1)

In [ ]:
new_apps['rating'] = 1

In [ ]:
new_apps = new_apps[['JobID','UserID','rating']]

In [ ]:
new_apps.UserID.unique().shape

(33402,)

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 237.3 kB/s eta 0:00:001m239.4 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 365.6 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2804143 sha256=0ba18fc5a08609b9facaf382405750443ae3bdc11f43f2b2c50de8185de77809
  Stored in directory: /home/abdollahimohammad/.cache/pip/wheels/df/e4/a6/7ad72453dd693f420b0c639bedeec34641738d11b55d8d9b84
Successfully built scikit-surprise

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import surprise
import os
from surprise import SVD
from surprise import KNNBasic, BaselineOnly, NormalPredictor
from surprise.model_selection import cross_validate
from collections import defaultdict
from surprise import accuracy
import base64
import time
import json
from surprise.model_selection import train_test_split

In [ ]:
import random
neg = np.array([[np.ones(30)*i, random.sample((np.arange(33402)[np.isin(np.arange(33402), new_apps[new_apps['JobID'] == i].UserID) == False]).tolist(), 30)] for i in range(1000)])

In [ ]:
for i in range(1000):
    for j in range(30):
        new_apps = pd.concat([new_apps, pd.DataFrame({'JobID':[i], 'UserID':[int(neg[i][1][j])], 'rating':[0]})])

In [ ]:
new_apps

,JobID,UserID,rating
50,0,0,1
52,1,1,1
59,2,1,1
85,0,1,1
94,3,1,1
...,...,...,...
0,999,29885,0
0,999,15992,0
0,999,12052,0
0,999,19010,0


In [ ]:
new_apps = new_apps.drop([0,1,2], axis=1).dropna()

KeyError: '[0, 1, 2] not found in axis'

In [ ]:
reader = surprise.Reader(rating_scale=(0, 1))
data = surprise.Dataset.load_from_df(new_apps, reader)
trainset, testset = train_test_split(data, test_size=.1)
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

print('rmse', accuracy.rmse(predictions))
print('mae', accuracy.mae(predictions))

RMSE: 0.4314
rmse 0.4314466177959897
MAE:  0.3558
mae 0.3558104312570575
